ПАКЕТ SURPRISE

используйте данные MovieLens 1M

можно использовать любые модели из пакета

получите RMSE на тестовом сете 0.87 и ниже

Комментарий преподавателя :

В ДЗ на датасет 1М может не хватить RAM. Можно сделать на 100K. Качество RMSE предлагаю считать на основе CrossValidation (5 фолдов), а не отложенном датасете.

Файл Практика.ipynb лежит в слаке в чате aml7ds9

In [2]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

In [3]:
users = pd.read_table('users.dat', sep='::', header=None, names=['user_id', 'gender', 'age', 'occupation', 'zip'])
users.head()

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


,user_id,gender,age,occupation,zip
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [4]:
users.shape

(6040, 5)

In [5]:
movies = pd.read_table('movies.dat', sep='::', header=None, names=['movie_id', 'title', 'genres'])
movies.head()

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [6]:
movies.shape

(3883, 3)

In [7]:
ratings = pd.read_table('ratings.dat', sep='::', header=None, names=['user_id', 'movie_id', 'rating', 'timestamp'])
ratings.head()

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [8]:
ratings.shape

(1000209, 4)

In [19]:
df_user1 = ratings.merge(movies, on='movie_id', how='left')

In [20]:
df_user1.head()

,user_id,movie_id,rating,timestamp,title,genres
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
1,1,661,3,978302109,James and the Giant Peach (1996),Animation|Children's|Musical
2,1,914,3,978301968,My Fair Lady (1964),Musical|Romance
3,1,3408,4,978300275,Erin Brockovich (2000),Drama
4,1,2355,5,978824291,"Bug's Life, A (1998)",Animation|Children's|Comedy


In [21]:
df_user1.shape

(1000209, 6)

In [28]:
from surprise import Dataset, Reader, KNNBasic
from surprise.model_selection import train_test_split
from surprise import accuracy

In [24]:
df_for_surpise = df_user1[['user_id', 'movie_id', 'rating']]

In [25]:
df_for_surpise.columns = ['uid', 'iid', 'rating']

In [26]:
df_for_surpise.head()

,uid,iid,rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5


In [29]:
reader = Reader(rating_scale=(0.5, 5))

In [30]:
dataset = Dataset.load_from_df(df_for_surpise, reader)

In [44]:
trainset, testset = train_test_split(dataset, test_size=0.3)

In [45]:
algo = KNNBasic(k=500, sim_options={'name': 'cosine', 'user_based': True})

In [46]:
algo.fit(trainset)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [47]:
predictions = algo.test(testset)

In [48]:
accuracy.mse(predictions)

MSE: 0.9514


0.9513984082600071